### Get stock prices data from polygon API

In [1]:
#!pip install -r requirements.txt

In [2]:
from stockify import * 
import datetime as dt 
from polygon import RESTClient
from polygon.rest import models
from http import HTTPStatus
    
API_key = 'PuJHla6_U_GDHcNE3XSpk7L2gCCpdJ9J'
client = RESTClient(API_key)

In [3]:
## Put the API key (if we do pay) into ENV_VARS
current_date = dt.datetime.now().date()
stocks_tickers = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=True)

stocks_requirments = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=False)

prices_dfs = []
dividends_dfs = []
DP_ratio_dfs = []

stocks_tickers=['ARR']
for st in stocks_tickers:
    
    ##Prices process
    #try:
        p_raw_response = get_p_process_by_ticker(API_key,
                                            client,
                                            st,
                                            get_daily_time_buckets(max_lookback_years=2,
                                                                    current_is_timestamp=False,
                                                                    frequency='daily',
                                                                    include_weekend=False),
                                            stock_prices_frequency= 'daily')
        
        pdf = get_jsonList_prices_into_dataframe(get_serialized_price_raw_into_json(p_raw_response,st),
                                    ['close'])
    # If exceed request per minute
    #except (BadResponse) as e:
    #    continue
    #    print(e)

        prices_dfs.append(pdf)
    
    ##Dividends process
    
        d_raw_response = get_top_x_dividends_by_ticker(ticker=st,
                                                    client=client,
                                                    API_latency_secs=1.5,
                                                    n_API_calls=144
                                                    )
        

        ddf = get_jsonList_dividends_into_dataframe(DivjsonList = d_raw_response,
                                                    measurable_time_variables= ['pay_date'])
                      

        dividends_dfs.append(ddf)
        
        DP_ratio_df = get_dividend_price_ratios(ticker=st,
                                                prices_df=pdf,
                                                dividends_df=ddf)
        
        DP_ratio_dfs.append(DP_ratio_df)


if len(prices_dfs)>0: 
    prices_dfs = pd.concat(prices_dfs)
    
if len(dividends_dfs)>0: 
    dividends_dfs = pd.concat(dividends_dfs)
    
if len(DP_ratio_dfs)>0: 
    DP_ratio_dfs = pd.concat(DP_ratio_dfs)

In [4]:
DP_ratio_dfs

,stock,close,converted_utc_timestamp,converted_utc_timestamp_back_shifted,n_paid_dividens_12_m,sum_paid_dividens_12_m,DP_ratio
0,ARR,5.59,2022-12-28 05:00:00,2021-12-28 05:00:00,13.0,1.3,0.232558
1,ARR,5.77,2022-12-27 05:00:00,2021-12-27 05:00:00,12.0,1.2,0.207972
2,ARR,5.89,2022-12-23 05:00:00,2021-12-23 05:00:00,12.0,1.2,0.203735
3,ARR,5.85,2022-12-22 05:00:00,2021-12-22 05:00:00,12.0,1.2,0.205128
4,ARR,5.77,2022-12-21 05:00:00,2021-12-21 05:00:00,12.0,1.2,0.207972
...,...,...,...,...,...,...,...
499,ARR,10.62,2021-01-05 05:00:00,2020-01-06 05:00:00,10.0,1.2,0.112994
500,ARR,10.61,2021-01-04 05:00:00,2020-01-05 05:00:00,10.0,1.2,0.113101
501,ARR,10.79,2020-12-31 05:00:00,2020-01-01 05:00:00,10.0,1.2,0.111214
502,ARR,10.74,2020-12-30 05:00:00,2019-12-31 05:00:00,10.0,1.2,0.111732


In [5]:
prices_dfs

,stock,converted_utc_timestamp,converted_utc_date,close
0,ARR,2020-12-29 05:00:00,2020-12-29,10.71
1,ARR,2020-12-30 05:00:00,2020-12-30,10.74
2,ARR,2020-12-31 05:00:00,2020-12-31,10.79
3,ARR,2021-01-04 05:00:00,2021-01-04,10.61
4,ARR,2021-01-05 05:00:00,2021-01-05,10.62
...,...,...,...,...
499,ARR,2022-12-21 05:00:00,2022-12-21,5.77
500,ARR,2022-12-22 05:00:00,2022-12-22,5.85
501,ARR,2022-12-23 05:00:00,2022-12-23,5.89
502,ARR,2022-12-27 05:00:00,2022-12-27,5.77


In [11]:
dividends_dfs[(dividends_dfs['pay_date']<='2022-12-28')&(dividends_dfs['pay_date']>='2021-12-28')].cash_amount.sum()

1.3000000000000003